In [13]:
# 1 - Anger
# 2 - Fear
# 3 - Sad
# 4 - Joy

import numpy as np
import pandas as p
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import precision_score, classification_report, recall_score, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import re
import warnings
warnings.filterwarnings('ignore')

In [6]:
trainanger=p.read_csv("/Users/juilee81/Desktop/EI-reg-En-train/EI-reg-En-anger-train.txt", delimiter='\t')
trainfear=p.read_csv("/Users/juilee81/Desktop/EI-reg-En-train/EI-reg-En-fear-train.txt", delimiter='\t')
trainsad=p.read_csv("/Users/juilee81/Desktop/EI-reg-En-train/EI-reg-En-sadness-train.txt", delimiter='\t')
trainjoy=p.read_csv("/Users/juilee81/Desktop/EI-reg-En-train/EI-reg-En-joy-train.txt", delimiter='\t')

In [7]:
train = p.concat([trainanger,trainfear,trainsad,trainjoy])

In [16]:
train['hashtags'] = train['Tweet'].apply(lambda x: re.findall(r"#(\w+)", x))
train['users'] = train['Tweet'].apply(lambda x: re.findall(r"@(\w+)", x))
train

,ID,Tweet,Affect Dimension,Intensity Score,hashtags,users,hashtag
0,2017-En-10264,@xandraaa5 @amayaallyn6 shut up hashtags are c...,anger,0.562,[offended],"[xandraaa5, amayaallyn6]",[offended]
1,2017-En-10072,it makes me so fucking irate jesus. nobody is ...,anger,0.750,[],[],[]
2,2017-En-11383,Lol Adam the Bull with his fake outrage...,anger,0.417,[],[],[]
3,2017-En-11102,@THATSSHAWTYLO passed away early this morning ...,anger,0.354,[],[THATSSHAWTYLO],[]
4,2017-En-11506,@Kristiann1125 lol wow i was gonna say really?...,anger,0.438,[],[Kristiann1125],[]
...,...,...,...,...,...,...,...
1611,2017-En-30316,Watch this amazing live.ly broadcast by @kana_...,joy,0.558,[musically],[kana_blender],[musically]
1612,2017-En-31092,Watching @melissamccarthy in #Spy she's one of...,joy,0.780,"[Spy, theboss, hilarious]",[melissamccarthy],"[Spy, theboss, hilarious]"
1613,2017-En-31037,Could not be happier!!,joy,0.885,[],[],[]
1614,2017-En-31270,@strictlysimilak something about English spark...,joy,0.360,[],[strictlysimilak],[]


In [25]:
train['Affect Dimension'] = train['Affect Dimension'].map(dict(anger=1, fear=2, sadness=3, joy=4))
y=train['Affect Dimension'].values
X=train['Tweet'].values

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(4971,) (4971,) (2131,) (2131,)


In [27]:
unique, counts = np.unique(y_train, return_counts=True)
print(unique)
print(counts)

[1 2 3 4]
[1199 1567 1064 1141]


#### MultiNomial Naive Bayes

In [36]:
unigram_count_vectorizer = CountVectorizer(encoding='latin-1', binary=False, min_df=5,
                                           stop_words='english', token_pattern='[a-z][a-z][a-z]+')
X_train_vec = unigram_count_vectorizer.fit_transform(X_train)
X_test_vec = unigram_count_vectorizer.transform(X_test)

nb_clf= MultinomialNB()
nb_clf.fit(X_train_vec,y_train)
y_pred = nb_clf.predict(X_test_vec)
cm=confusion_matrix(y_test, y_pred, labels=[1,2,3,4])
print(cm)
target_names = ['1','2','3','4']
print(classification_report(y_test, y_pred, target_names=target_names,digits = 3))

[[407  39  39  17]
 [ 37 572  52  24]
 [ 46  60 346  17]
 [ 20  24  29 402]]
              precision    recall  f1-score   support

           1      0.798     0.811     0.804       502
           2      0.823     0.835     0.829       685
           3      0.742     0.738     0.740       469
           4      0.874     0.846     0.860       475

    accuracy                          0.810      2131
   macro avg      0.809     0.807     0.808      2131
weighted avg      0.811     0.810     0.811      2131



In [29]:
# bigram Count vectorizer, set minimum document frequency to 5
bigram_count_vectorizer = CountVectorizer(encoding='latin-1', binary=False, min_df=5, ngram_range = (1,2),
                                           stop_words='english', token_pattern='[a-z][a-z][a-z]+')
X_train_vec = bigram_count_vectorizer.fit_transform(X_train)
X_test_vec = bigram_count_vectorizer.transform(X_test)

nb_clf= MultinomialNB()
nb_clf.fit(X_train_vec,y_train)
y_pred = nb_clf.predict(X_test_vec)
cm=confusion_matrix(y_test, y_pred, labels=[1,2,3,4])
print(cm)
target_names = ['1','2','3','4']
print(classification_report(y_test, y_pred, target_names=target_names))

[[408  40  38  16]
 [ 36 574  51  24]
 [ 46  62 344  17]
 [ 24  25  32 394]]
              precision    recall  f1-score   support

           1       0.79      0.81      0.80       502
           2       0.82      0.84      0.83       685
           3       0.74      0.73      0.74       469
           4       0.87      0.83      0.85       475

    accuracy                           0.81      2131
   macro avg       0.81      0.80      0.80      2131
weighted avg       0.81      0.81      0.81      2131



In [37]:
# trigram Count vectorizer, set minimum document frequency to 5
trigram_count_vectorizer = CountVectorizer(encoding='latin-1', binary=False, min_df=5, ngram_range = (1,3),
                                           stop_words='english', token_pattern='[a-z][a-z][a-z]+')
X_train_vec = trigram_count_vectorizer.fit_transform(X_train)
X_test_vec = trigram_count_vectorizer.transform(X_test)

nb_clf= MultinomialNB()
nb_clf.fit(X_train_vec,y_train)
y_pred = nb_clf.predict(X_test_vec)
cm=confusion_matrix(y_test, y_pred, labels=[1,2,3,4])
print(cm)
target_names = ['1','2','3','4']
print(classification_report(y_test, y_pred, target_names=target_names))

[[412  41  38  11]
 [ 36 576  51  22]
 [ 47  62 347  13]
 [ 27  30  35 383]]
              precision    recall  f1-score   support

           1       0.79      0.82      0.80       502
           2       0.81      0.84      0.83       685
           3       0.74      0.74      0.74       469
           4       0.89      0.81      0.85       475

    accuracy                           0.81      2131
   macro avg       0.81      0.80      0.80      2131
weighted avg       0.81      0.81      0.81      2131



#### Linear SVC

In [38]:
# unigram Count vectorizer, set minimum document frequency to 5
unigram_count_vectorizer = CountVectorizer(encoding='latin-1', binary=False, min_df=5, 
                                           stop_words='english', token_pattern='[a-z][a-z][a-z]+')
X_train_vec = unigram_count_vectorizer.fit_transform(X_train)
X_test_vec = unigram_count_vectorizer.transform(X_test)

svm_clf = LinearSVC(C=0.2)
svm_clf.fit(X_train_vec,y_train)
y_pred = svm_clf.predict(X_test_vec)
cm=confusion_matrix(y_test, y_pred, labels=[1,2,3,4])
print(cm)
target_names = ['1','2','3','4']
print(classification_report(y_test, y_pred, target_names=target_names,digits = 3))

[[410  53  25  14]
 [ 32 596  41  16]
 [ 40  58 361  10]
 [  9  35  17 414]]
              precision    recall  f1-score   support

           1      0.835     0.817     0.826       502
           2      0.803     0.870     0.835       685
           3      0.813     0.770     0.791       469
           4      0.912     0.872     0.891       475

    accuracy                          0.836      2131
   macro avg      0.841     0.832     0.836      2131
weighted avg      0.837     0.836     0.836      2131



In [39]:
# bigram Count vectorizer, set minimum document frequency to 5
bigram_count_vectorizer = CountVectorizer(encoding='latin-1', binary=False, min_df=5, ngram_range = (1,2),
                                           stop_words='english', token_pattern='[a-z][a-z][a-z]+')
X_train_vec = bigram_count_vectorizer.fit_transform(X_train)
X_test_vec = bigram_count_vectorizer.transform(X_test)

svm_clf = LinearSVC(C=0.2)
svm_clf.fit(X_train_vec,y_train)
y_pred = svm_clf.predict(X_test_vec)
cm=confusion_matrix(y_test, y_pred, labels=[1,2,3,4])
print(cm)
target_names = ['1','2','3','4']
print(classification_report(y_test, y_pred, target_names=target_names,digits = 3))

[[411  51  25  15]
 [ 34 594  42  15]
 [ 40  57 362  10]
 [  9  37  14 415]]
              precision    recall  f1-score   support

           1      0.832     0.819     0.825       502
           2      0.804     0.867     0.834       685
           3      0.817     0.772     0.794       469
           4      0.912     0.874     0.892       475

    accuracy                          0.836      2131
   macro avg      0.841     0.833     0.836      2131
weighted avg      0.838     0.836     0.836      2131

